In [1]:
import pandas as pd
import mysql.connector

In [ ]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database="amazon_data",
    connect_timeout=1000
)

cursor = mydb.cursor()
print("connection created")
only_changing_issue_ids = []
weekly_counts = {}
monthly_counts = {}
changed_issue_ids_dict = {}

In [ ]:
# Fetch data from the database and create dataframes for each issue ID
def fetch_data_grouped_by_issue_ids():
    query = (
        "SELECT "
        "   `Status`, `Notes`, `Issue type`, `Issue ID`, `Infraction Subtype Code`, "
        "   `Reversal Invoice #`, `Charge Invoice #`, `Financial charge`, `scrape_date` "
        "FROM csv_data_scrape_date "
        "WHERE `Issue ID` IN ("
        "   SELECT `Issue ID` "
        "   FROM csv_data_scrape_date "
        "   GROUP BY `Issue ID` "
        "   HAVING COUNT(*) > 2"
        ") "
    )
    print("Fetching data from DB")

    data_frames_dict = {}  # Dictionary to store dataframes for each issue ID
    data_frames_dict_without_scrape = {}  # Dictionary without scrape_date column
    cursor.execute(query)
    all_data = cursor.fetchall()
    column_names = [col[0] for col in cursor.description]

    print("query executed")
    for row in all_data:
        issue_id_index = column_names.index('Issue ID')
        issue_id = row[issue_id_index]
        if issue_id not in data_frames_dict:
            data_frames_dict[issue_id] = []

        data_frames_dict[issue_id].append(row)

        # Create a row without the 'scrape_date' column
        row_without_scrape = [row[i] for i, col_name in enumerate(column_names) if col_name != 'scrape_date']
        if issue_id not in data_frames_dict_without_scrape:
            data_frames_dict_without_scrape[issue_id] = []

        data_frames_dict_without_scrape[issue_id].append(row_without_scrape)

    mydb.close()

    return data_frames_dict, data_frames_dict_without_scrape


In [ ]:
all_data_chunks, all_data_chunks_without_date = fetch_data_grouped_by_issue_ids()
print("Total issue ids : ", len(all_data_chunks), len(all_data_chunks_without_date))

In [ ]:
cols_to_check = [
    "`Status`", "`Notes`", "`Issue type`", "`Issue ID`", "`Infraction Subtype Code`",
    "`Reversal Invoice #`", "`Charge Invoice #`", "`Financial charge`", "`scrape_date`"
]

cols_to_check_without_date = [
    "`Status`", "`Notes`", "`Issue type`", "`Issue ID`", "`Infraction Subtype Code`",
    "`Reversal Invoice #`", "`Charge Invoice #`", "`Financial charge`"
]


In [ ]:
def process_chunks(data_frames_dict):

    with open('real_data_output_GT2_complete.txt', 'w') as log_file:
    
        count = 1
    
        column_total_change_sums = {col: 0 for col in cols_to_check_without_date}
        column_total_days_to_change = {col: 0 for col in cols_to_check_without_date}

        for index, rows_list in data_frames_dict.items():
            
            if (count % 20000 == 0):
                print("Reached iteration : ", count)        
            count = count + 1

            returned_specific_issue_data = pd.DataFrame(rows_list, columns=cols_to_check)

            column_avg_change_sums = {col: 0 for col in cols_to_check_without_date}
            column_change_days = {col: [] for col in cols_to_check_without_date}

            for j in range(1, len(returned_specific_issue_data) ):
            
                prev_row = returned_specific_issue_data.iloc[j - 1]
                current_row = returned_specific_issue_data.iloc[j]

                for col in cols_to_check_without_date:
            
                    if col != '`scrape_date`':  # Exclude 'scrape_date' from column change calculations
            
                        if prev_row[col] != current_row[col]:
                        
                            column_avg_change_sums[col] += 1
                            days_difference = (pd.to_datetime(current_row['`scrape_date`']) - pd.to_datetime(prev_row['`scrape_date`'])).days
                            column_change_days[col].append((prev_row[col], current_row[col], days_difference))

            total_changes = sum(column_avg_change_sums.values())

            if total_changes > 0:

                only_changing_issue_ids.append(index)
                print("************************************************", file=log_file)
                print("Changed columns for id : ", index, " Rows : ", len(rows_list), "\n", file=log_file)
                log_file.flush()

                # Print the column name, previous and next values, and total changes
                for col in cols_to_check_without_date:

                    if col != '`scrape_date`':  # Exclude 'scrape_date' from output
                        print(f"{col} ({column_avg_change_sums[col]}):", file=log_file)
                    
                        if col in column_change_days and len(column_change_days[col]) > 0:
                            for prev_val, next_val, days_diff in column_change_days[col]:
                                print(f"\t{prev_val} -> {next_val} (Change took {days_diff} days)", file=log_file)
                        
                        log_file.flush()

                        # Accumulate the column change sums and days to change
                        column_total_change_sums[col] += column_avg_change_sums[col]
                        if len(column_change_days[col]) > 0:
                            column_total_days_to_change[col] += sum(days_diff for _, _, days_diff in column_change_days[col]) / len(column_change_days[col])

                print("\nTotal changes:", total_changes, file=log_file)
                log_file.flush()

                # Display average days to change for each column for the current ID
                print("\nAverage Days to Change for Each Column (ID:", index, "):", file=log_file)
                log_file.flush()
                for col in cols_to_check_without_date:
                    if col != '`scrape_date`':  # Exclude 'scrape_date' from output
                        avg_days_to_change = column_total_days_to_change[col] / len(column_change_days[col]) if len(column_change_days[col]) > 0 else 0
                        print("\t", col, ":", avg_days_to_change, " days to change", file=log_file)

        # Write total sum of changes for each column at the end of the file
        print("************************************************", file=log_file)
        print("Total Sum of Changes for Each Column:", file=log_file)
        for col, total_change_sum in column_total_change_sums.items():
            if col != '`scrape_date`':
                print(f"\t{col}:", total_change_sum, "changes", file=log_file)

In [ ]:
process_chunks(all_data_chunks)

In [ ]:
print("Only changing issue ids : ",len(only_changing_issue_ids))

In [ ]:
def filter_changing_issue_ids(data_frames_dict, issue_ids_to_filter):
    changed_issue_ids = {issue_id: data_frames_dict[issue_id] for issue_id in issue_ids_to_filter}
    return changed_issue_ids

In [ ]:
changed_issue_ids_dict = filter_changing_issue_ids(all_data_chunks, only_changing_issue_ids)
print(len(changed_issue_ids_dict))

In [ ]:
def weekly_report(changed_issue_ids_dict):
    weekly_counts = {}  # Dictionary to store weekly counts of changes

    count = 1

    for index, rows_list in changed_issue_ids_dict.items():
        if count % 20000 == 0:
            print("Iteration reached:", count)
        count = count + 1

        returned_specific_issue_data = pd.DataFrame(rows_list, columns=cols_to_check)

        # Convert the 'scrape_date' column to datetime format
        returned_specific_issue_data['`scrape_date`'] = pd.to_datetime(returned_specific_issue_data['`scrape_date`'])

        # Group the data by week
        grouped = returned_specific_issue_data.groupby(returned_specific_issue_data['`scrape_date`'].dt.to_period('W'))

        # Iterate through each week and its corresponding grouped data
        for week, group_data in grouped:
            week_str = str(week)
            
            # Create a dictionary entry for the current week if it doesn't exist
            if week_str not in weekly_counts:
                weekly_counts[week_str] = {col: 0 for col in cols_to_check_without_date}
            
            # Calculate the changes in columns for the current week
            column_changes = {col: 0 for col in cols_to_check_without_date}
            
            for idx in range(1, len(group_data)):  # Start from index 1 to skip the first row
                prev_row = group_data.iloc[idx - 1]
                row = group_data.iloc[idx]
                for col in cols_to_check_without_date:
                    if col != '`scrape_date`' and prev_row[col] != row[col]:
                        column_changes[col] += 1
            
            # Update the weekly counts with the changes for this week
            for col in cols_to_check_without_date:
                weekly_counts[week_str][col] += column_changes[col]

    with open('real_data_output_GT2_complete.txt', 'a') as log_file:
        print("************************************************", file=log_file)

        # Calculate and print the sum of changes for each week
        for week, col_changes in weekly_counts.items():
            weekly_total_changes = sum(col_changes.values())
            print("\nTotal Changes for week", week, ":", weekly_total_changes, file=log_file)

        # Calculate and print the total sum of changes across all weeks
        total_changes = {col: 0 for col in cols_to_check_without_date}
        for col_changes in weekly_counts.values():
            for col, changes in col_changes.items():
                if col != '`scrape_date`':
                    total_changes[col] += changes
        total_sum_changes = sum(total_changes.values())
        print("\nTotal Sum of Changes across all weeks:", total_sum_changes, file=log_file)

        total_weeks = len(weekly_counts)
        print("Total number of weeks:", total_weeks, file=log_file)

        if total_weeks > 0:
            print("Average changes for all weeks:", total_sum_changes / total_weeks, file=log_file)
        else:
            print("Average changes for all weeks: 0", file=log_file)

In [ ]:
weekly_report(changed_issue_ids_dict)

In [ ]:
def monthly_report(changed_issue_ids_dict):

    monthly_counts = {}  # Dictionary to store monthly counts of changes

    count = 1

    for index, rows_list in changed_issue_ids_dict.items():

        if count % 20000 == 0:
            print("Iteration reached:", count)
        count = count + 1

        returned_specific_issue_data = pd.DataFrame(rows_list, columns=cols_to_check)

        # Convert the 'scrape_date' column to datetime format
        returned_specific_issue_data['`scrape_date`'] = pd.to_datetime(returned_specific_issue_data['`scrape_date`'])

        # Group the data by month
        grouped = returned_specific_issue_data.groupby(returned_specific_issue_data['`scrape_date`'].dt.to_period('M'))

        # Iterate through each month and its corresponding grouped data
        for month, group_data in grouped:
            month_str = str(month)
            
            # Create a dictionary entry for the current month if it doesn't exist
            if month_str not in monthly_counts:
                monthly_counts[month_str] = {col: 0 for col in cols_to_check_without_date}
            
            # Calculate the changes in columns for the current month
            column_change_sums = {col: 0 for col in cols_to_check_without_date}
            prev_row = None
            
            for idx, row in group_data.iterrows():
                if prev_row is not None:
                    for col in cols_to_check_without_date:
                        if col != '`scrape_date`' and prev_row[col] != row[col]:
                            column_change_sums[col] += 1
                prev_row = row
            
            # Update the monthly counts with the changes for this month
            for col in cols_to_check_without_date:
                monthly_counts[month_str][col] += column_change_sums[col]

    with open('real_data_output_GT2_complete.txt', 'a') as log_file:

        print("************************************************", file=log_file)

        # Calculate and print the sum of changes for each month
        for month, col_changes in monthly_counts.items():
            monthly_total_changes = sum(col_changes.values())
            print("\nTotal Changes for month", month, ":", monthly_total_changes, file=log_file)

        # Calculate and print the total sum of changes across all months
        total_changes = {col: 0 for col in cols_to_check_without_date}
        for col_changes in monthly_counts.values():
            for col, changes in col_changes.items():
                if col != '`scrape_date`':
                    total_changes[col] += changes
        total_sum_changes = sum(total_changes.values())
        print("\nTotal Sum of Changes across all months:", total_sum_changes, file=log_file)

        total_months = len(monthly_counts)
        print("Total number of months:", total_months, file=log_file)

        if total_months > 0:
            print("Average changes for all months:", total_sum_changes / total_months, file=log_file)
        else:
            print("Average changes for all months: 0", file=log_file)

In [ ]:
monthly_report(changed_issue_ids_dict)